# Hermit Fold 1

In [1]:
import os
from collections import Counter

In [2]:
import pandas as pd

In [3]:
from leaderboard.utils import rasa_utils

In [4]:
import importlib
importlib.reload(rasa_utils)

<module 'leaderboard.utils.rasa_utils' from '/Users/kabu/Documents/workspace/rasa2/leaderboard/utils/rasa_utils.py'>

## Load Data

In [5]:
data_path = "../../../../training-data/public/HERMIT/KFold_1"

In [6]:
!ls {data_path}

test  train


In [7]:
!head {data_path}/train/train.yml

version: "2.0"
nlu:
- intent: alarm_query
  examples: |
    - what alarms do i have set right now
    - checkout [today](date) alarm of meeting
    - report alarm settings
    - see see for me the alarms that you have set [tomorrow](date) [morning](timeofday)
    - is there an alarm for [ten am](time)
    - confirm the alarm time


In [8]:
data = {split: rasa_utils.load_nlu_data(os.path.join(hermit_path, split, f'{split}.yml'), domain_path = None) for split in ['train', 'test']}

NameError: name 'hermit_path' is not defined

In [ ]:
dfs = { 
    split: pd.DataFrame(
        [
            { 'text': message.get('text'), 
             'intent' : message.get('intent'), 
             'entities' : [entity['entity'] for entity in message.get('entities',[])],
            }
            for message in data[split].nlu_examples
        ]
    ) for split in data }

## Intents

In [ ]:
for split, df in dfs.items():
    print(f'\n{split}\n' + '-'*20)
    display(df['intent'].describe())

In [ ]:
dfs['train']['intent'].value_counts().hist()
dfs['test']['intent'].value_counts().hist(color = "orange")

In [ ]:
dfs['train']['intent'].value_counts(ascending=True)[:10]

In [ ]:
dfs['train']['intent'].value_counts(ascending=False)[:10]

## Entities

### Entities per Message

In [ ]:
for df in dfs.values():
    df['num_entities'] = df['entities'].apply(len)

In [ ]:
dfs['train']['num_entities'].hist(figsize=(15,5))
dfs['test']['num_entities'].hist(color = "orange")

In [ ]:
dfs['train']['num_entities'].value_counts(normalize=True)*100

In [ ]:
dfs['train']['num_entities'].value_counts()

In [ ]:
dfs['test']['num_entities'].value_counts()

### Messages per Entity

In [ ]:
def count_messages_per_entity(df) -> pd.DataFrame:
    entities_counter = Counter()
    for entities in df['entities']:
        entities_counter.update(entities)

    entities_counter_df = pd.DataFrame.from_dict(entities_counter,orient='index').reset_index()
    entities_counter_df.columns = ['entity', 'message_count']
    entities_counter_df.sort_values('message_count', inplace=True)
    return entities_counter_df

In [ ]:
message_per_entity_counts = {split: count_messages_per_entity(df) for split, df in dfs.items()}

In [ ]:
message_per_entity_counts['train']['message_count'].hist(figsize=(15,5), bins=100)
message_per_entity_counts['test']['message_count'].hist(figsize=(15,5), bins=50, color = 'orange')

In [ ]:
message_per_entity_counts['test']['message_count'].hist(figsize=(15,5), bins=50, color = 'orange')

In [ ]:
message_per_entity_counts['train']['message_count'].head(10)

In [ ]:
message_per_entity_counts['train']['message_count'].tail(10)

In [ ]:
message_per_entity_counts['train']['message_count'].head(10)

In [ ]:
message_per_entity_counts['test']['message_count'].tail(10)